In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
import torchvision
import pickle as pkl
from attack import attack, test_model, parse_param
import random
device = "cuda" if torch.cuda.is_available() else "cpu"


In [2]:
def setup_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True


setup_seed(3407)


In [3]:
from datasets import load_cifar10, load_cifar100
from models.resnet import load_cifar10_resnet50, load_cifar100_resnet50
model = load_cifar100_resnet50()


In [4]:
all_param_names = list()
for name, param in model.named_parameters():
    if not "bn" in name and not "shortcut.1" in name:
        all_param_names.append(name)


In [5]:
all_param_names = all_param_names[:-2]


In [6]:
train_loaders, test_dataloaders, train_dataloader_all, test_dataloader_all = load_cifar100()
all_totals = list()
for i in range(100):
    all_totals.append(attack(train_loaders[i], all_param_names, load_cifar100_resnet50,
                      train_dataloader_all, alpha=0.00001, num_steps=4, op="add"))


Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 2/2 [00:03<00:00,  1.93s/it]


0.0031813300848007203


100%|██████████| 2/2 [00:00<00:00,  3.20it/s]


0.01804310417175293


...

0.03505203628540039


100%|██████████| 2/2 [00:00<00:00,  3.32it/s]


0.39028611755371095


100%|██████████| 2/2 [00:00<00:00,  3.28it/s]


1.9754710693359374


In [7]:
pkl.dump(all_totals, open("weights/totals.pkl", "wb"))


In [20]:
thre = 0.07
net = load_cifar100_resnet50()
param_remove = dict()
for param in all_param_names:
    param_remove[param] = None
for i in range(len(all_totals)):
    totals = all_totals[i]
    totals = [totals[param] for param in all_param_names]
    param_weights = [eval("net." + parse_param(param) + ".cpu().detach().numpy()")
                     for param in all_param_names]
    combine = [np.abs(total * weight)
               for total, weight in zip(totals, param_weights)]
    combine = np.array(combine)
    combine_flatten = np.concatenate(
        [combine_.flatten() for combine_ in combine], axis=0)
    threshold = np.sort(combine_flatten)[
        ::-1][int(len(combine_flatten) * thre)]
    for idx, param in enumerate(all_param_names):
        if param_remove[param] is None:
            param_remove[param] = combine[idx] > threshold
        else:
            t = combine[idx] > threshold
            param_remove[param] = param_remove[param] | t


C:\Users\Zhang\AppData\Local\Temp\ipykernel_27276\1051719498.py:13: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  combine = np.array(combine)


In [21]:
temp = 0
all_num = 0
for param in param_remove:
    temp += param_remove[param].sum()
    all_num += param_remove[param].size
    print(param, param_remove[param].mean())


conv1.weight 0.9936342592592593
layer1.0.conv1.weight 0.883544921875
layer1.0.conv2.weight 0.6957736545138888
layer1.0.conv3.weight 0.76568603515625
layer1.0.shortcut.0.weight 0.80279541015625
layer1.1.conv1.weight 0.65869140625
layer1.1.conv2.weight 0.7159016927083334
layer1.1.conv3.weight 0.73040771484375
layer1.2.conv1.weight 0.66961669921875
layer1.2.conv2.weight 0.6825900607638888
layer1.2.conv3.weight 0.53460693359375
layer2.0.conv1.weight 0.8699951171875
layer2.0.conv2.weight 0.6604682074652778
layer2.0.conv3.weight 0.7593841552734375
layer2.0.shortcut.0.weight 0.655487060546875
layer2.1.conv1.weight 0.345794677734375
layer2.1.conv2.weight 0.5167914496527778
layer2.1.conv3.weight 0.6204986572265625
layer2.2.conv1.weight 0.4469451904296875
layer2.2.conv2.weight 0.5791219075520834
layer2.2.conv3.weight 0.6329803466796875
layer2.3.conv1.weight 0.540374755859375
layer2.3.conv2.weight 0.6033528645833334
layer2.3.conv3.weight 0.5842437744140625
layer3.0.conv1.weight 0.8491668701171875

In [22]:
temp / all_num


0.4880513827815582

In [23]:
with torch.no_grad():
    net = load_cifar100_resnet50()
    preds, labels = test_model(net, test_dataloader_all)
    print("原始准确率", (preds.argmax(-1) == labels).mean())


原始准确率 0.7929


In [24]:
with torch.no_grad():
    net = load_cifar100_resnet50()
    for param in all_param_names:
        param_ = parse_param(param)
        try:
            exec("net." + param_ + "[~param_remove[param]] = 0")
        except:
            exec("net." + param_ + "[~param_remove[param],:] = 0")
    preds, labels = test_model(net, test_dataloader_all)
    print("现在准确率", (preds.argmax(-1) == labels).mean())


现在准确率 0.4417


In [25]:
with torch.no_grad():
    net = load_cifar100_resnet50()
    for param in all_param_names:
        param_ = parse_param(param)
        keep_rate = param_remove[param].sum() / param_remove[param].size
        weight_flatten = eval(
            "net." + param_ + ".cpu().detach().numpy()").flatten()
        threshold = np.sort(weight_flatten)[int(
            len(weight_flatten) * (1 - keep_rate))]
        try:
            exec("net." + param_ +
                 "[eval('net.' + param_ + '.cpu().detach().numpy()') < threshold] = 0")
        except:
            exec("net." + param_ +
                 "[eval('net.' + param_ + '.cpu().detach().numpy()') < threshold,:] = 0")
    preds, labels = test_model(net, test_dataloader_all)
    print("对比实验准确率", (preds.argmax(-1) == labels).mean())


对比实验准确率 0.01
